# Q&A
使用するpdfファイルはpdfsの下に格納

In [1]:
#インポートするファイルまとめ

import os
import requests
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from genai.extensions.langchain import LangChainInterface
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from genai.model import Credentials, Model
from genai.schemas import GenerateParams, ModelType
from langchain.vectorstores.faiss import FAISS
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain import PromptTemplate, LLMChain

/Users/makyua452144/anaconda3/envs/watsonx_demo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/makyua452144/anaconda3/envs/watsonx_demo/lib/python3.11/site-packages/genai/schemas/__init__.py:10: DeprecationWarning: 
The class ModelType is being deprecated.
Please replace any reference to ModelType by its model id string equivalent.
Example :
  ModelType.FLAN_T5 becomes "google/flan-t5-xxl"

  from genai.schemas.models import ModelType


# 調整できるパラメーター
- chunk_liit: チャンクに含めることができる最大文字数
- chunk_overlap: AIが完全に文脈を見失わないように、chunkにまたがる文字数
- min_new_tokens: 生成された応答のトークンの最小数
- max_new_tokens: 生成された応答のトークンの最大数
- repetition_penalty: 生成したテキストに基づいて、新しいトークンに課すペナルティ
- search_kwargs: k(返されるドキュメントの数)、score_threshold(リトリーバーから返されるドキュメントの最小関連数)、fetch_k(MMRに渡すドキュメントの数)

In [9]:
#設定する箇所
chunk_limit = 700
chunk_overlap = 10
min_new_tokens=3
max_new_tokens=1500
repetition_penalty=1.1
search_kwargs={'k': 5, 'fetch_k': 50, 'score_threshold': 0.8}

# APIのkeyを挿入
api_key = "pak-aiAAErKRTfrLuK3Gl0p7-HOvMnJsU4UwmFf9wLJCaiA"

In [10]:
#変更しない

pdf_folder_path = './pdfs'
# print(os.listdir(pdf_folder_path))

#LLMの準備
api_endpoint = "https://bam-api.res.ibm.com/v1/"

#ベクターストアの場所
vec = "article_" +  str(chunk_limit) + "_" + str(chunk_overlap)
persist_directory="vec/" + vec

creds = Credentials(api_key=api_key, api_endpoint=api_endpoint)

# PDFのローディング
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)] #ローダー定義
documents = [docu.load() for docu in loaders]
new_documents = ""
for i in range(len(documents)):
    new_documents += documents[i][0].page_content

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

if vec in os.listdir("vec/"):
    #再利用
    index = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
else:
    
    #チャンクの分割
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_limit, chunk_overlap=chunk_overlap)
    pdf_text = text_splitter.split_text(new_documents)
    
    # ドキュメントの読みこみ
    index = Chroma.from_texts(pdf_text, embeddings, persist_directory=persist_directory)
    
params_qa = GenerateParams(
    decoding_method="greedy",
    min_new_tokens=min_new_tokens,
    max_new_tokens=max_new_tokens,
    stream=False,
    repetition_penalty=repetition_penalty,
).dict() 

llm_qa = LangChainInterface(model='meta-llama/llama-2-70b-chat', credentials=creds, params=params_qa)

#リトリーバーの設定(ユーザーから質問を受け取り、ベクターストアに投げかけ、関連コードを返してくれるところ)
qa_chain = RetrievalQA.from_chain_type(llm=llm_qa, 
                                    chain_type="stuff", 
                                    retriever=index.as_retriever(search_kwargs=search_kwargs), 
                                    input_key="question")

In [11]:
def generate_qa(pdf_text, question):
    template = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

----------------
{ground_text}
----------------
                
----------------
Question: {question}
----------------

You must answer in Japanese.

Helpful answer in Japanese:"""
    prompt = PromptTemplate(
        template=template, 
        input_variables=["ground_text", "question"]
    )
    llm_chain = LLMChain(prompt=prompt, llm=llm_qa, verbose=True)
    answer = llm_chain.predict(ground_text=pdf_text, question=question)
     
    return answer

In [12]:
def regenerate_qa(doc, q, ans):
    template = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

----------------
{ground_text}
----------------
                
----------------
Question: {question}
----------------

You must answer in Japanese.

Helpful answer in Japanese:""" + ans
    
    prompt = PromptTemplate(
        template=template, 
        input_variables=["ground_text", "question"]
    )
    llm_chain = LLMChain(prompt=prompt, llm=llm_qa, verbose=True)
    answer = llm_chain.predict(ground_text=doc, question=q)
     
    return answer

In [13]:
latest_doc = ""
latest_q = None
latest_ans = None

# ボタンがクリックされたときに実行される関数
def execute_func2(history):
    global latest_ans, latest_doc, latest_q  # グローバル変数を参照

    if latest_ans is not None:
        answer = regenerate_qa(latest_doc, latest_q, latest_ans)
        # func2の結果をUI上で表示
        history = history + [("", answer)]
        return history

# Chat UIの起動
def add_text(history, text):
    history = history + [(text, None)]
    return history, ""

def bot(history):
    global latest_ans, latest_doc, latest_q  # グローバル変数を参照

    latest_q = history[-1][0]
    
    retriever = index.as_retriever()
    docs = retriever.get_relevant_documents(query=latest_q)
    # print(f"\n参考箇所: \n {docs}")
    latest_doc = ""
    for d in docs:
        latest_doc += d.page_content
        latest_doc += "\n"

    latest_ans = generate_qa(latest_doc, latest_q)
    print(f"\n回答: \n {latest_ans}")

    retriever = index.as_retriever()
    docs = retriever.get_relevant_documents(query=latest_q)
    print(f"\n参考箇所: \n {docs}")
    
    history[-1][1] = latest_ans
    return history

In [14]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chat with PDF").style(height=600)
    with gr.Row():
        with gr.Column(scale=0.6):
            txt = gr.Textbox(
                show_label=False,
                placeholder="Type your question and press enter",
            ).style(container=False)
        with gr.Column(scale=0.4):
            # ボタンを追加。ボタンがクリックされたときに`execute_func2`が呼ばれる。
            btn = gr.Button(text="Execute Func2", elem_id="execute_button").style(height=20, width=120)
            btn.click(execute_func2, inputs=chatbot, outputs=chatbot)
            
    txt.submit(add_text, [chatbot, txt], [chatbot, txt]).then(
        bot, chatbot, chatbot
    )

demo.launch()

/var/folders/1z/n4cgf80j1y5d5vlv2ws02_q40000gn/T/ipykernel_70356/3548103617.py:2: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  chatbot = gr.Chatbot([], elem_id="chat with PDF").style(height=600)
/var/folders/1z/n4cgf80j1y5d5vlv2ws02_q40000gn/T/ipykernel_70356/3548103617.py:8: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  ).style(container=False)
/var/folders/1z/n4cgf80j1y5d5vlv2ws02_q40000gn/T/ipykernel_70356/3548103617.py:11: GradioUnusedKwargWarning: You have unused kwarg parameters in Button, please remove them: {'text': 'Execute Func2'}
  btn = gr.Button(text="Execute Func2", elem_id="execute_button").style(height=20, width=120)
/var/folders/1z/n4cgf80j1y5d5vlv2ws02_q40000gn/T/ipykernel_70356/3548103617.py:11: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  btn = gr.Button(

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/Users/makyua452144/anaconda3/envs/watsonx_demo/lib/python3.11/site-packages/gradio/analytics.py:46: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if StrictVersion(latest_pkg_version) > StrictVersion(current_pkg_version):
/Users/makyua452144/anaconda3/envs/watsonx_demo/lib/python3.11/site-packages/gradio/analytics.py:46: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if StrictVersion(latest_pkg_version) > StrictVersion(current_pkg_version):
/Users/makyua452144/anaconda3/envs/watsonx_demo/lib/python3.11/site-packages/chromadb/utils/read_write_lock.py:29: DeprecationWarning: notifyAll() is deprecated, use notify_all() instead
  self._read_ready.notifyAll()




> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

----------------
ます。

ドロップダウンメニューから選択するだけで新しいモデルをお試しいただけます。スタジオの詳

細については、こちらをご覧ください。

未来への展望

基盤モデルは、AIの展望を変えつつあり、進歩はさらに加速しています。私たちIBMは、この急

速に進化する分野の最前線でイノベーションを真の企業価値に変えることをご支援していきま

す。

以上

当報道資料は、2023年5月9日（現地時間）にIBM Corporationが発表したブログの抄訳です。原

文はこちらをご参照ください。

Release Categories

AI

United States — English

Products & Solutions

Learn about

About IBM

Top products & platforms

What is Hybrid Cloud?

Annual report

Industries

What is Artificial intelligence?

Career opportunities

Artificial intelligence

What is Cloud Computing?

Corporate social responsibility

Blockchain

What is Kubernetes?

Diversity & inclusion

Business operations

What are Containers?

Industry analyst reports
Cloud computing

What is DevOps?

Investor relations

Data & Analytics

Wha

/Users/makyua452144/anaconda3/envs/watsonx_demo/lib/python3.11/site-packages/chromadb/utils/read_write_lock.py:29: DeprecationWarning: notifyAll() is deprecated, use notify_all() instead
  self._read_ready.notifyAll()
/Users/makyua452144/anaconda3/envs/watsonx_demo/lib/python3.11/site-packages/chromadb/utils/read_write_lock.py:29: DeprecationWarning: notifyAll() is deprecated, use notify_all() instead
  self._read_ready.notifyAll()




> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

----------------
ます。

ドロップダウンメニューから選択するだけで新しいモデルをお試しいただけます。スタジオの詳

細については、こちらをご覧ください。

未来への展望

基盤モデルは、AIの展望を変えつつあり、進歩はさらに加速しています。私たちIBMは、この急

速に進化する分野の最前線でイノベーションを真の企業価値に変えることをご支援していきま

す。

以上

当報道資料は、2023年5月9日（現地時間）にIBM Corporationが発表したブログの抄訳です。原

文はこちらをご参照ください。

Release Categories

AI

United States — English

Products & Solutions

Learn about

About IBM

Top products & platforms

What is Hybrid Cloud?

Annual report

Industries

What is Artificial intelligence?

Career opportunities

Artificial intelligence

What is Cloud Computing?

Corporate social responsibility

Blockchain

What is Kubernetes?

Diversity & inclusion

Business operations

What are Containers?

Industry analyst reports
まなタスクにおいて高い推論効率と性能レベルを提供します。Slate は、エンコーダーのみの

RoBERTa ベースのモデル群を指し、生成型ではありま

/Users/makyua452144/anaconda3/envs/watsonx_demo/lib/python3.11/site-packages/chromadb/utils/read_write_lock.py:29: DeprecationWarning: notifyAll() is deprecated, use notify_all() instead
  self._read_ready.notifyAll()


# グリッド検索